<a href="https://colab.research.google.com/github/lodelabuelo/Wav2Lip/blob/master/Wav2Lip_SIMPLIFICADO_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goal: Make anyone speak anything (LipSync)

* Github: https://github.com/Rudrabha/Wav2Lip
* Paper: https://arxiv.org/abs/2008.10010
*Original notebook: https://colab.research.google.com/drive/1tZpDWXz49W6wDcTprANRGLo2D_EbD5J8?usp=sharing





In [ ]:
#@title <h1>PASO 1: Instalar Wav2Lip</h1>
#@markdown * Darle al PLAY para correr este código y esperar a qué termine

!rm -rf /content/sample_data
!mkdir /content/sample_data

# !git clone https://github.com/Rudrabha/Wav2Lip
!git clone https://github.com/lodelabuelo/wav2lip_fonamor.git

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/wav2lip_fonamor/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd wav2lip_fonamor && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/wav2lip_fonamor/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python


#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

%cd /
from ghc.l_ghc_cf import l_ghc_cf
%cd content

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr


from IPython.display import HTML
from base64 import b64encode
def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

from IPython.display import clear_output

Cloning into 'wav2lip_fonamor'...
remote: Enumerating objects: 379, done.
remote: Counting objects: 100% (379/379), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 379 (delta 214), reused 358 (delta 203), pack-reused 0
Receiving objects: 100% (379/379), 500.65 KiB | 19.26 MiB/s, done.
Resolving deltas: 100% (214/214), done.
--2022-08-18 17:18:54--  https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA
Resolving iiitaphyd-my.sharepoint.com (iiitaphyd-my.sharepoint.com)... 13.107.136.9, 13.107.138.9
Connecting to iiitaphyd-my.sharepoint.com (iiitaphyd-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 435801865 (416M) [application/octet-stream]
Saving to: ‘/content/wav2lip_fonamor/checkpoints/wav2lip_gan.pth’

/content/wav2lip_fo 100%[===================>] 415.61M   153MB/s    in 2.7s    

2022-08-18 17:18:58 (1

# LipSync on Your Video File

In [ ]:
#@title Paso 2: Donde sucede la magia
#@markdown En Premiere exportar el fragmento de video que queres modificar y nombrarlo <b>input_vid.mp4</b> <br>
#@markdown IMPORTANTE: Debe tener durante TODA la duración del video un rostro que no se bloquee con ningún objeto ni nada. Recomiendo clips cortos :) <br><br>
#@markdown Hacer lo mismo con el audio y nombrarlo <b>input_audio.wav</b> <br>
#@markdown IMPORTANTE: El audio debe contener solo la voz objetivo. Cuanto más hablada mejor. Si queres que la boca se cierre, tiene que haber true silencio. Si vas a hacerlo para una canción o similar, mejor grabarte aparte la voz limpia y lo menos cantada posible pero a tempo con la canción. <br><br>
#@markdown Poner <b>input_vid.mp4</b> e <b>input_audio.wav</b> en la carpeta <b>sample_data</b> que está oculta en el icono de carpetita de la izquierda <br><br>
#@markdown <b>RECIEN AHORA LE DAS PLAY</b> <br><br>
#@markdown Esperar a que haga su proceso y el resultado lo encontrás en: wav2lip_fonamor/results/result_voice.mp4 <br><br>
#@markdown <b>Si notas algun error tipo como que clipea la boca, jugar con estos valores, sino dejarlos como están</b>
pad_top =  0#@param {type:"integer"}
pad_bottom =  10#@param {type:"integer"}
pad_left =  0#@param {type:"integer"}
pad_right =  0#@param {type:"integer"}
rescaleFactor =  1#@param {type:"integer"}
nosmooth = True #@param {type:"boolean"}

if nosmooth == False:
  !cd wav2lip_fonamor && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "../sample_data/input_vid.mp4" --audio "../sample_data/input_audio.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor
else:
  !cd wav2lip_fonamor && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "../sample_data/input_vid.mp4" --audio "../sample_data/input_audio.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor --nosmooth

#Preview output video
clear_output()
print("Final Video Preview")
print("Dowload this video from", '/content/wav2lip_fonamor/results/result_voice.mp4')
showVideo('/content/wav2lip_fonamor/results/result_voice.mp4')


Final Video Preview
Dowload this video from /content/wav2lip_fonamor/results/result_voice.mp4
